In [5]:
import re
import torch
from torch import nn
from torch.nn import functional as F
from torch.nn.utils.rnn import pack_padded_sequence
from typing import Any, cast

class PackPaddedLSTM(nn.Module):
    """LSTM model with pack_padded layers."""

    def __init__(
        self,
        vocab_size: int = 60,
        embedding_size: int = 128,
        output_size: int = 18,
        hidden_size: int = 32,
    ):
        super().__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(vocab_size, embedding_size)
        self.lstm = nn.LSTM(embedding_size, self.hidden_size, num_layers=1)  # type: ignore[no-untyped-call] # noqa: E501
        self.hidden2out = nn.Linear(self.hidden_size, output_size)
        self.dropout_layer = nn.Dropout(p=0.2)

    def forward(self, batch: torch.Tensor, lengths: torch.Tensor) -> torch.Tensor:
        hidden1 = torch.ones(1, batch.size(-1), self.hidden_size, device=batch.device)
        hidden2 = torch.ones(1, batch.size(-1), self.hidden_size, device=batch.device)
        embeds = self.embedding(batch)
        packed_input = pack_padded_sequence(embeds, lengths)
        _, (ht, _) = self.lstm(packed_input, (hidden1, hidden2))
        output = self.dropout_layer(ht[-1])
        output = self.hidden2out(output)
        output = F.log_softmax(output, dim=1)
        return cast(torch.Tensor, output)

In [12]:
model = PackPaddedLSTM()
from torchinfo import summary
summary(model)

Layer (type:depth-idx)                   Param #
PackPaddedLSTM                           --
├─Embedding: 1-1                         7,680
├─LSTM: 1-2                              20,736
├─Linear: 1-3                            594
├─Dropout: 1-4                           --
Total params: 29,010
Trainable params: 29,010
Non-trainable params: 0

In [14]:
m_list = []
for _, m in model.named_modules():
    m_list.append(m)
print(m_list)

[PackPaddedLSTM(
  (embedding): Embedding(60, 128)
  (lstm): LSTM(128, 32)
  (hidden2out): Linear(in_features=32, out_features=18, bias=True)
  (dropout_layer): Dropout(p=0.2, inplace=False)
), Embedding(60, 128), LSTM(128, 32), Linear(in_features=32, out_features=18, bias=True), Dropout(p=0.2, inplace=False)]


In [8]:
from parsers import ModelParser
parser = ModelParser(model)
architecture = parser.parse(verbose=False)

In [9]:
out_path = "LSTM.json"
parser.export(out_path)

'LSTM.json\\architecture.json'

In [1]:
from models.relaynet.relay_net import ReLayNet
# from models.hrnet.model4c import HRNetRefine
# from models.hrnet.channelconfig import get_cfg_defaults
import json
import re
import torch
import inspect

from parsers import ModelParser

In [12]:
model_params = {
    'num_channels': 1,
    'num_filters': 64,
    'kernel_h': 7,
    'kernel_w': 3,
    'kernel_c': 1,
    'stride_conv': 1,
    'pool': 2,
    'stride_pool': 2,
    'num_class': 3,
    'epochs': 6
}

In [8]:
configs = get_cfg_defaults()
model = HRNetRefine(configs)
parser = ModelParser(model)
architecture = parser.parse(verbose=False)

In [ ]:
pytest_list = []
for i in range(len(layers_json)):
    pytest_list.append(layers_json[i]['l_type'])
return layers_json, pytest_list

In [15]:
print(len(architecture))

29


In [32]:
def model_output(model):
    m_list = []
    for _, m in model.named_modules():
        if len(list(m.named_modules()))==1:
            m_list.append(m)
    lower_case = []
    for i in m_list:
        module_name = re.match("^\w+",str(i)).group()
        lower_case.append(module_name)
    return m_list, lower_case

In [34]:
m_list, lower_case = model_output(model)
assert lower_case == pytest_list

In [16]:
pytest_list = []
for i in range(len(architecture)):
    pytest_list.append(architecture[i]['l_type'])

In [35]:
model = ReLayNet(model_params)
parser = ModelParser(model)
architecture = parser.parse(verbose=False)
out_path = "json_file/architecture.json"
parser.export(out_path)


'json_file\\architecture.json\\architecture.json'

In [22]:
model_params = {
'num_channels': 1,
'num_filters': 64,
'kernel_h': 7,
'kernel_w': 3,
'kernel_c': 1,
'stride_conv': 1,
'pool': 2,
'stride_pool': 2,
'num_class': 3,
'epochs': 6
}
# Relaynet
model = ReLayNet(model_params)
parser = ModelParser(model)
layers_json = parser.parse_model()
with open("model.json", "w") as f:
    json.dump(layers_json, f)
# HRNET
# configs = get_cfg_defaults()
# model = HRNetRefine(configs)
# parser = ModelParser(model)
# layers_json = parser.parse_model()
# with open("sample.json", "w") as outfile:
#     outfile.write(layers_json)

In [24]:
len(layers_json)

29

In [ ]:
### Pytest

In [ ]:
dictionary = json.load(open("sample.json","r"))
pytest_list = []
for i in range(len(dictionary)):
    pytest_list.append(dictionary[i]['l_type'])


In [ ]:
m_list = []
for name, m in model.named_modules():
    if len(list(m.named_modules()))==1:
        m_list.append(m)

print(len(m_list))
# print(m_list)
lower_case = []
for i in m_list:
    module_name = re.match("^\w+",str(i)).group()
    lower_case.append(module_name.lower())

29


In [ ]:
assert lower_case == pytest_list